In [1]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams["figure.figsize"] = (10,8)

import warnings
warnings.filterwarnings('ignore')
from datetime import datetime 

In [2]:
df = pd.read_csv("datos/bikes.csv", index_col=0)
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [4]:
df.sample(10)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
207,208,27-07-2018,autumn,0,7,0,3,1,1,31.775000,34.53335,40.2917,12.292021,755,3901,4656
92,93,03-04-2018,summer,0,4,0,0,0,1,15.511653,18.93835,48.0000,12.208271,1651,1598,3249
399,400,04-02-2019,spring,1,2,0,6,0,2,10.830847,13.63625,77.9583,8.167032,384,2448,2832
113,114,24-04-2018,summer,0,4,0,0,0,2,23.848347,27.58815,81.0833,12.875725,1710,2481,4191
666,667,29-10-2019,winter,1,10,0,1,1,3,18.040000,21.97000,88.0000,23.999400,2,20,22
281,282,09-10-2018,winter,0,10,0,0,0,1,22.174153,26.19915,72.7500,4.251150,2397,3114,5511
609,610,02-09-2019,autumn,1,9,0,0,0,2,28.563347,32.45000,81.5000,4.292744,2613,3197,5810
128,129,09-05-2018,summer,0,5,0,1,1,1,21.832500,26.26230,58.8750,11.792000,664,3698,4362
623,624,16-09-2019,autumn,1,9,0,0,0,1,23.780000,28.15625,57.0000,6.042281,2166,5167,7333
76,77,18-03-2018,spring,0,3,0,5,1,1,22.140000,26.35045,52.5217,15.478139,884,2355,3239


In [6]:
#df_spring = df[df['season'] == 'spring']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


In [8]:
df['dteday'] = pd.to_datetime(df['dteday'], format= '%d-%m-%Y')

In [17]:
df.sample(20)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
583,584,2019-08-07,autumn,1,8,0,2,1,2,30.169153,34.88105,70.3750,7.832836,1278,5995,7273
289,290,2018-10-17,winter,0,10,0,1,1,1,21.900847,25.53625,57.9583,11.750393,713,3857,4570
627,628,2019-09-20,autumn,1,9,0,4,1,1,22.413347,26.60960,61.8333,7.917189,939,6781,7720
4,5,2018-01-05,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
457,458,2019-04-03,summer,1,4,0,2,1,1,19.133347,23.07415,39.6667,6.708911,1348,5424,6772
68,69,2018-03-10,spring,0,3,0,4,1,3,15.952731,19.28340,0.0000,17.545759,46,577,623
474,475,2019-04-20,summer,1,4,0,5,1,1,21.593347,25.78875,69.4583,10.041357,1340,5950,7290
698,699,2019-11-30,winter,1,11,0,5,1,1,12.231653,16.19335,64.9583,3.917544,362,5306,5668
212,213,2018-08-01,autumn,0,8,0,1,1,1,31.638347,35.16460,55.0833,10.500039,729,3537,4266
157,158,2018-06-07,summer,0,6,0,2,1,1,29.007500,32.79750,59.7917,12.583136,763,4070,4833


In [ ]:
def estaciones(dataframe, columna):

    if 

In [ ]:
#df['month'] = df['dteday'].dt.month_name()